# Playground

A safe space for experimentation.

In [2]:
%load_ext autoreload
%autoreload 2

import os
import schedule
import requests
import numpy as np
import pandas as pd
from etl_task import ETLTask
from datetime import datetime, timedelta
from etl_db_manager import ETLDataBaseManager
from utility import base64encode_obj, base64decode_obj

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load Data

In [3]:
def fetch_data_airnoise(sensor_id:str, time_start:datetime, time_end:datetime):
    """ Fetches air/noise data from the API
        at: https://data.smartdublin.ie/sonitus-api. 
        @param sensor_id: Serial number of the sensor.
        @param time_start: Time range starting point.
        @param time_end: Time range ending point.
        @return: Data from API or [].
    """
    data = []
    try:
        res = requests.post(f"https://data.smartdublin.ie/sonitus-api/api/data", json={ 
            'username': "dublincityapi",
            'password': "Xpa5vAQ9ki",
            'monitor': sensor_id,
            'start': time_start.timestamp(),
            'end': time_end.timestamp()
        })
        print("[DEBUG] res =", res)
        data = res.json()
    except Exception as e:
          raise print(f'Failed to fetch air noise data from source: {e}')
    return data

In [4]:
def load_data_noise():
    """ Loads last 30 mins worth of noise data 
        from multiple noise sensors.
    """
    dt_now = datetime.now() # Time now.
    dt_past = dt_now - timedelta(minutes=30) # Time 30 mins ago.
    # Fetch data from multiple sensors.
    data_sensors = {"01749":[], "01508":[], "10118":[], "01548":[], "10115":[]}
    for sensor in data_sensors.keys():
        data_sensors[sensor] = fetch_data_airnoise(
            sensor_id=sensor, 
            time_start=dt_past, 
            time_end=dt_now
        )
    return data_sensors

## Schedule Load Save

In [5]:
def load_data_bikes():
    """ Loads last 30 min snapshot of dublin bike stands. """
    import requests
    data = []
    try:
        res = requests.get(f"https://data.smartdublin.ie/dublinbikes-api/last_snapshot")  
        data = res.json()
    except Exception as e:
          data = []
          print(f'Failed to fetch dublin bikes data from source: {e}')
          raise Exception(f'Failed to fetch dublin bikes data from source: {e}')
    return data

def transform_data_bikes(data):
    """
    Transforms bikes data to be in a desireable format for saving.
    @param data: Data to be transformed.
    """
    import pytz
    import pandas as pd
    from datetime import datetime

    df = pd.DataFrame(data)
    df['usage_percent'] = df['available_bikes']/df['bike_stands']
    df['usage_percent'] = df['usage_percent'].round(2)
    df['status'] = df['status'].str.lower()
    df = df[[
        'station_id', 'bike_stands', 
        'available_bikes', 'usage_percent', 
        'last_update', 'status'
    ]]
    print("Bike data transformed. Usage % computed and last update made lowercase.")

    return df.to_dict(orient='records')

def save_data_bikes(data):
    """ 
    Saves given data to csv file. 
    @param data: Data to be saved.
    """
    import requests
    try:
        res = requests.post(
            url="http://localhost:8000/bikes/snapshot/", 
            json={'snapshot':data}
        )
        print(res.json()['message'])
    except Exception as e:
        print(f'Failed to save bikes data. {e}')
        raise Exception(f'Failed to save bikes data. {e}')

In [6]:
# data = load_data_bikes()
# data = transform_data_bikes(data)
# save_data_bikes(data)

In [7]:
# Toy data load and save functions.
def load_data(): 
    print(f'Loaded data.')
    return [1, 2, 3]

def transform_data(data):
    data_transformed = data * 10 # Deliberate error to showcase logging.
    print(f'Transformed data {data} into {data_transformed}.')
    return data_transformed

def save_data(data): 
    print(f'Saved data {data}.')

In [8]:
def make_post_request(url, data={}):
    """
    Makes a post request.
    @param url: URL to post to.
    @data: Request body data.
    @return: Response.
    """
    response = requests.post(url, params=data)
    return {'status': response.status_code, 'message': response.json()['message']}

def make_get_request(url, data={}):
    """
    Makes a post request.
    @param url: URL to post to.
    @data: Request body data.
    @return: Response.
    """
    response = requests.get(url, params=data)
    return {'status': response.status_code, 'message': response.text}

def make_delete_request(url, data={}):
    """
    Makes a delete request.
    @param url: URL to post to.
    @data: Request body data.
    @return: Response.
    """
    response = requests.delete(url, params=data)
    return {'status': response.status_code, 'message': response.text}

def make_put_request(url, task_name, new_values):
    """
    Makes a put request.
    @param url: URL to post to.
    @data: Request body data.
    @return: Response.
    """
    response = requests.put(
        url=url, 
        params={"task_name": task_name}, 
        json=new_values
    )
    return {'status': response.status_code, 'message': response.json()}

In [9]:
# data = load_data_bikes()
# data = load_data_bikes(data)
# save_data_bikes(data)

In [11]:
# Create a new task.
task = ETLTask(
    name='bikes',
    fun_data_load=load_data_bikes,
    fun_data_transform=transform_data_bikes,
    fun_data_save=save_data_bikes,
    repeat_time_unit='seconds',
    repeat_interval=10
)
task_str = base64encode_obj(task)
make_post_request(url="http://127.0.0.1:8003/task", data={"task_str": task_str})

{'status': 200, 'message': 'Success. Task created and scheduled bikes.'}

In [12]:
# Create a new task.
task = ETLTask(
    name='test',
    fun_data_load=load_data,
    fun_data_transform=transform_data,
    fun_data_save=save_data,
    repeat_time_unit='seconds',
    repeat_interval=3
)
task_str = base64encode_obj(task)
make_post_request(url="http://127.0.0.1:8003/task", data={"task_str": task_str})

{'status': 200, 'message': 'Success. Task created and scheduled test.'}

In [16]:
# Delete a task.
make_delete_request(url="http://127.0.0.1:8003/task", data={'task_name': 'bikes'})

{'status': 200,
 'message': '{"status":200,"message":"Success. Deleted task bikes.","data":[]}'}

In [15]:
# Start scheduler.
make_get_request(url="http://127.0.0.1:8003/start_scheduler")

{'status': 200,
 'message': '{"status":200,"message":"Scheduler started.","data":[]}'}

In [17]:
# Stop scheduler.
make_get_request(url="http://127.0.0.1:8003/stop_scheduler")

{'status': 200,
 'message': '{"status":200,"message":"Scheduler stopped.","data":[]}'}

In [187]:
# Stop task
make_put_request(
    url="http://127.0.0.1:8003/task/stop", 
    task_name="test",
    new_values={}
)

{'status': 200,
 'message': {'status': 200,
  'message': 'Success. Task test has been stopped.',
  'data': []}}